# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import pandas as pd

In [3]:
# Load data
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [4]:
# Define features and target variable
X = fires_dt.drop('area', axis=1)
y = fires_dt['area']

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Define preprocessing for numeric and categorical features
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Preprocessor 1: Scaling only
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder

# Define your numeric and categorical feature lists
numeric_features = ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Preprocessor 2: Scaling and applying Power Transformation
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('scaler', StandardScaler()), ('power', PowerTransformer())]), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

# Baseline regressor
baseline_model = Ridge()

# Advanced regressor
advanced_model = RandomForestRegressor()

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [8]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline(steps=[('preprocessing', preproc1), ('regressor', baseline_model)])

In [9]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(steps=[('preprocessing', preproc2), ('regressor', baseline_model)])

In [10]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(steps=[('preprocessing', preproc1), ('regressor', advanced_model)])

In [11]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline(steps=[('preprocessing', preproc2), ('regressor', advanced_model)])
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [12]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grids
param_grid_A = {'regressor__alpha': [0.1, 1.0, 10.0]}


# Grid search
grid_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error')


# Fit models
grid_A.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error')

In [13]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grids
param_grid_B = {'regressor__alpha': [0.1, 1.0, 10.0]}

# Grid search
grid_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error')


# Fit models
grid_B.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler()),
                                                                                         ('power',
                                                                                          PowerTransformer())]),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error')

In [14]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grids
param_grid_C = {'regressor__n_estimators': [100, 200], 'regressor__max_depth': [10, 20]}


# Grid search
grid_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error')


# Fit models
grid_C.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', RandomForestRegressor())]),
             param_grid={'regressor__max_depth': [10, 20],
                         'regressor__n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

In [15]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameter grids
param_grid_D = {'regressor__n_estimators': [100, 200], 'regressor__max_depth': [10, 20]}

# Grid search
grid_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error')

# Fit models
grid_D.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler()),
                                                                                         ('power',
                                                                                          PowerTransformer())]),
                                                                         ['ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', RandomForestRegressor())]),
             param_grid={'regressor__max_depth': [10, 20],
                         'regressor__n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

# Evaluate

+ Which model has the best performance?

In [17]:
# Best model evaluation
print(f"Best model A: {grid_A.best_score_}")
print(f"Best model B: {grid_B.best_score_}")
print(f"Best model C: {grid_C.best_score_}")
print(f"Best model D: {grid_D.best_score_}")

# Select the best model
best_model = max([grid_A, grid_B, grid_C, grid_D], key=lambda g: g.best_score_)

Best model A: -4278.655355069459
Best model B: -4279.955377282284
Best model C: -4768.967396700597
Best model D: -4575.162135031686


# Export

+ Save the best performing model to a pickle file.

In [18]:
import joblib

# Save the best performing model
joblib.dump(best_model, 'best_forest_fire_model.pkl')

['best_forest_fire_model.pkl']

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

Here’s a simpler paraphrase of the text:

**Choose an Example and Explain Important Features**  
 We can use the SHAP force plot we created to see which features are most important for that example's prediction. In the force plot, each feature has a SHAP value showing how much it affects the prediction. Red bars mean a feature increases the prediction, while blue bars mean it decreases it. The features with the highest SHAP values are the most influential for that prediction.

For example, if a feature's SHAP value is around +3, it greatly increases the prediction, whereas a value around -3 greatly decreases it. By looking at these SHAP values, we can determine which features change the prediction from the starting point to the final prediction of 18.61. Features that have a big effect on the prediction are crucial for understanding why the model made that specific prediction.

**Overall Importance of Features**  
To find out which features are generally important across the training set, we calculate the average absolute SHAP values for all features. Features with higher average SHAP values have a bigger impact on predictions, while those with lower values are less important.

**Most Important Features**  
The features that have the highest average absolute SHAP values are:
- num__dc: 7.50
- cat__month_sep: 5.83
- num__dmc: 5.75
- num__temp: 4.92
- num__coord_x: 3.71  

These features greatly influence the model's predictions, indicating that the model relies heavily on them for accuracy.

**Least Important Features**  
The features that contribute the least to predictions, with the lowest average SHAP values, are:
- cat__month_may: 0.0025
- cat__month_nov: 0.0031
- cat__month_feb: 0.0067
- cat__month_jan: 0.0159
- num__ffmc: 0.022  

These features have minimal impact on the model’s predictions, suggesting they may not be crucial for overall accuracy.

**Analysis**  
Overall, numerical features like num__dc, num__dmc, and num__temp are very important, indicating they are key factors for the model. Certain categorical features, like specific months, also play a significant role. Conversely, some months with fewer events, like May and November, contribute very little.

**Removing Features**  
We should consider removing features with consistently low SHAP values since they don’t help the model much and might add unnecessary complexity.

**Features to Consider Removing**  
The least important features we might remove include:
- cat__month_may
- cat__month_nov
- cat__month_feb
- cat__month_jan  
- num__ffmc  

Other low-importance categorical features, like cat__day_tue or cat__day_thu, might also be removed, but we need more analysis to see if they consistently have low value.

**Why Remove These Features?**  
Removing unimportant features can simplify the model, improve its interpretability, reduce overfitting, and lower computational costs without greatly affecting performance.

**Testing the Impact of Feature Removal**  
To check if removing features improves performance, we can follow these steps:

1. **Create a Reduced Feature Set**: Remove the low-importance features and train the model again with the remaining features.
2. **Evaluate Model Performance**:
   - **Cross-Validation**: Test the reduced model with cross-validation and compare the results to the original model.
   - **Test Set Performance**: Evaluate the reduced model on the test set and compare metrics like RMSE (Root Mean Square Error) or MAE (Mean Absolute Error) to the original model.
3. **Compare Results**:
   - If the reduced model has similar or better accuracy (e.g., lower RMSE or MAE), it suggests that the removed features weren’t very useful.
   - If the reduced model shows more consistent performance across different data sets, it indicates a more robust model with less risk of overfitting.
4. **Conduct Statistical Tests**: We could perform a paired t-test on the performance scores to check if any differences are statistically significant.
5. **Monitor Interpretability**: Check the SHAP values for the reduced model to ensure that important features still make sense and that the model remains understandable.

By using this process, we can systematically assess the effect of removing less important features and confirm that the model's performance remains the same or improves.

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.